<a href="https://colab.research.google.com/github/JavalVyas2000/Estimation-of-RUL/blob/main/Estimation_of_RUL_using_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing all the prerequisited for connecting with the drive.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Installing the python libraries that we may use
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
data3=pd.read_excel('gdrive/MyDrive/RUL data/B0005_Discap.xlsx')

In [ ]:
cycle_no=[]
p=int(data3['Cycle'][data3.shape[0]-1])
for i in range(p):
  cycle_no.append(i+1)
Discap=[None]*len(cycle_no)
for i in range(data3.shape[0]-1):
  Discap[int(data3['Cycle'][i])]=data3['Discap'][i]

In [ ]:
# creating a dataframe and interpolating the missing values

df5 = pd.DataFrame(Discap,cycle_no)
df5.interpolate(method ='linear', limit_direction ='forward',axis=0,inplace=True)
df5.dropna(inplace=True)
plt.figure(figsize=(5,2.5))
plt.plot(df5)
plt.xlabel('Cycle_no')
plt.ylabel('Discap')
plt.title('B0005')

In [ ]:
# Scaling the data 
numpy.random.seed(7)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset5 = scaler.fit_transform(df5)

In [ ]:
# Spliting into train and test
train_size5 = int(len(dataset5) * 0.67)
test_size5 = int(len(dataset5)) - train_size5
train5, test5 = dataset5[0:train_size5,:], dataset5[train_size5:len(dataset5),:]
print(len(train5), len(test5))

In [ ]:
# Creating a look_back dataframe 
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

look_back = 1
trainX5, trainY5 = create_dataset(train5, look_back)
testX5, testY5 = create_dataset(test5, look_back)

In [ ]:
# Reshaping the data to fit the LSTM architecture
trainX5 =trainX5.reshape(trainX5.shape[0],trainX5.shape[1] , 1)
testX5 = testX5.reshape(testX5.shape[0],testX5.shape[1] , 1)

In [ ]:
# Creating LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
model5=Sequential()
model5.add(LSTM(4,input_shape=(1,1)))
model5.add(Dense(1))
model5.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
# fitting the data into the model
model5.fit(trainX5,trainY5,validation_data=(testX5,testY5),epochs=100,batch_size=1,verbose=2)

In [ ]:
# predicting the values 
import tensorflow as tf
train_predict5=model5.predict(trainX5)
test_predict5=model5.predict(testX5)

trainPredictPlot = numpy.empty_like(dataset5)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(train_predict5)+look_back, :] = train_predict5

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset5)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict5)+(look_back*2)+1:len(dataset5)-1, :] = test_predict5

plt.figure(figsize=(10,5))
plt.plot(scaler.inverse_transform(trainPredictPlot))
plt.plot(scaler.inverse_transform(testPredictPlot))
plt.plot(df5)
plt.xlabel('Cycle_no')
plt.ylabel('Discap')
plt.title('B0005')
plt.show()

In [ ]:
# evaluation metrics - RMSE, MSE, MAPE, MAE
testScore = math.sqrt(mean_squared_error(df5[:len(test_predict5)], test_predict5))
print('Test Score: %.2f RMSE' % (testScore))
testScore21 = mean_squared_error(df5[:len(test_predict5)], test_predict5)
print('Test Score21: %.4f MSE' % (testScore21))
pred=np.array(test_predict5)
actual=np.array(df5[:len(test_predict5)])
MAPE2=np.mean(np.abs((actual-pred)/actual))*100
print('Test Score: %.4f MAPE' % (MAPE2))
MAE2=np.mean(np.abs((actual-pred)))
print('Test Score: %.4f MAE' % (MAE2))